In [ ]:
# =====================================================
# 1. Instalación e Importación de Librerías Necesarias
# =====================================================
!pip install --quiet google-cloud-bigquery pandas matplotlib seaborn scikit-learn

try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from google.cloud import bigquery
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, PowerTransformer
    from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
    from sklearn.decomposition import PCA
    print("✅ Todas las librerías necesarias están instaladas.")
except ImportError as e:
    print(f"❌ Falta instalar alguna librería: {e}")


In [ ]:
# =====================================================
# 2. Cargar Variables Secretas en Google Colab
# =====================================================
from google.colab import userdata
import os

# Obtener variables secretas (si las configuraste en Google Colab)
PROJECT_ID = userdata.get('PROJECT_ID')  # ID de tu Proyecto de Google Cloud
DATASET_ID = "tracking_dataset"  # ⚠️ Cambia esto con el nombre de tu dataset en BigQuery
TABLE_ID = "tracking_data"  # ⚠️ Tabla que contiene los datos

# Verificar si las variables se cargaron correctamente
if not PROJECT_ID:
    raise ValueError("❌ No se encontró PROJECT_ID en Secrets.")

print("✅ Variables secretas cargadas correctamente.")

In [ ]:
# ================================
# 3. Autenticación en Google Cloud
# ================================
from google.colab import auth
auth.authenticate_user()

# Conectar con BigQuery
client = bigquery.Client(project=PROJECT_ID)
print(f"✅ Conexión establecida con BigQuery en el proyecto: {PROJECT_ID}")

In [ ]:
# =====================================================
# 4. Obtener Datos desde BigQuery
# =====================================================
query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`"
df = client.query(query).to_dataframe()

# Mostrar los primeros registros
display(df.head())


In [ ]:
# =====================================================
# 5. Ingeniería de Características
# =====================================================
# Copia del DataFrame original para la transformación
df_transformed = df.copy()

# -----------------------------------------------------
# 5.1 Generación de Nuevas Características
# -----------------------------------------------------
if 'battery_level' in df_transformed.columns:
    df_transformed['battery_low'] = (df_transformed['battery_level'] < 20).astype(int)

if 'signal_strength' in df_transformed.columns:
    df_transformed['signal_quality'] = pd.cut(df_transformed['signal_strength'],
                                              bins=[-120, -90, -70, 0],
                                              labels=["Baja", "Media", "Alta"])

# -----------------------------------------------------
# 5.2 Discretización (Binning)
# -----------------------------------------------------
if 'latitude' in df_transformed.columns and 'longitude' in df_transformed.columns:
    df_transformed['location_bin'] = (df_transformed['latitude'].astype(str) + "_" +
                                      df_transformed['longitude'].astype(str))

# -----------------------------------------------------
# 5.3 Codificación de Variables Categóricas
# -----------------------------------------------------
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['signal_quality', 'location_bin']

# Verificar que las columnas existen en el DataFrame antes de aplicar la codificación
categorical_cols = [col for col in categorical_cols if col in df_transformed.columns]

if categorical_cols:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

    encoded_features = encoder.fit_transform(df_transformed[categorical_cols])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))

    df_transformed = df_transformed.drop(columns=categorical_cols)
    df_transformed = pd.concat([df_transformed, encoded_df], axis=1)

    print(f"✅ Se han codificado las siguientes columnas: {categorical_cols}")
else:
    print("⚠️ No se encontraron columnas categóricas para codificar.")

# -----------------------------------------------------
# 5.4 Escalamiento de Variables Numéricas
# -----------------------------------------------------
scaler = StandardScaler()
numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()
df_transformed[numeric_cols] = scaler.fit_transform(df_transformed[numeric_cols])

# Mostrar las primeras filas después de la transformación
display(df_transformed.head())

In [ ]:
# =====================================================
# 6. Selección y Extracción de Características
# =====================================================

# -----------------------------------------------------
# 6.1 Eliminación de Características con Baja Varianza
# -----------------------------------------------------

# Filtrar solo columnas numéricas antes de aplicar VarianceThreshold
numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()

if numeric_cols:
    var_thresh = VarianceThreshold(threshold=0.01)

    # Aplicar VarianceThreshold solo a las columnas numéricas
    df_numeric_filtered = df_transformed[numeric_cols]
    df_high_variance = pd.DataFrame(var_thresh.fit_transform(df_numeric_filtered),
                                    columns=[col for i, col in enumerate(numeric_cols) if var_thresh.variances_[i] > 0.01])

    print(f"✅ Se han conservado {df_high_variance.shape[1]} características con suficiente varianza.")
    display(df_high_variance.head())
else:
    print("⚠️ No hay suficientes columnas numéricas para aplicar VarianceThreshold.")

# -----------------------------------------------------
# 6.2 Selección de Características por Importancia Estadística (ANOVA)
# -----------------------------------------------------
from sklearn.feature_selection import SelectKBest, f_classif

if 'status' in df_transformed.columns:
    # Filtrar solo columnas numéricas antes de aplicar SelectKBest
    numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()

    # Asegurar que la variable objetivo también es numérica
    if df_transformed['status'].dtype == 'object':
        df_transformed['status'] = df_transformed['status'].astype('category').cat.codes

    # Separar variables predictoras (X) y variable objetivo (y)
    X = df_transformed[numeric_cols].drop(columns=['status'], errors='ignore')
    y = df_transformed['status']

    if not X.empty:
        selector = SelectKBest(score_func=f_classif, k=min(5, X.shape[1]))  # Selecciona hasta 5 características
        X_selected = selector.fit_transform(X, y)

        selected_features = X.columns[selector.get_support()]
        df_selected = pd.DataFrame(X_selected, columns=selected_features)

        print(f"✅ Se han seleccionado las {df_selected.shape[1]} características más relevantes.")
        display(df_selected.head())
    else:
        print("⚠️ No hay suficientes características numéricas para aplicar SelectKBest.")

# -----------------------------------------------------
# 6.3 Reducción de Dimensionalidad con PCA
# -----------------------------------------------------
pca = PCA(n_components=5)
principal_components = pca.fit_transform(df_transformed[numeric_cols])

df_pca = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(5)])
display(df_pca.head())

# Visualizar la varianza explicada por los componentes principales
plt.figure(figsize=(8, 5))
plt.plot(range(1, 6), np.cumsum(pca.explained_variance_ratio_), marker="o", linestyle="--")
plt.xlabel("Número de Componentes")
plt.ylabel("Varianza Acumulada")
plt.title("Varianza Explicada por PCA")
plt.grid(True)
plt.show()

In [ ]:
# =====================================================
# 7. Subir los Datos Transformados a BigQuery
# =====================================================
from pandas_gbq import to_gbq

# Función para limpiar nombres de columnas y hacerlos válidos en BigQuery
def clean_column_names(df):
    df.columns = (df.columns
                  .str.replace(r'\W', '_', regex=True)  # Reemplaza cualquier carácter no alfanumérico con "_"
                  .str.lower()  # Convierte todo a minúsculas
                  .str[:290]  # Asegura que los nombres de columna no excedan 300 caracteres
                 )
    return df

# Aplicar limpieza de nombres de columnas
df_transformed = clean_column_names(df_transformed)

# Especificar la tabla de destino en BigQuery
table_id_transformed = f"{DATASET_ID}.tracking_data_transformed"

# Subir los datos transformados a BigQuery
to_gbq(df_transformed, destination_table=table_id_transformed, project_id=PROJECT_ID, if_exists="replace")

print(f"✅ Datos transformados cargados en BigQuery: {table_id_transformed}")

# Conclusión

Aquí generamos un conjunto de datos sintéticos representativo para poder monitorear dispositivos en movimiento dentro de un área geográfica específica, en este caso Houston, TX. Este dataset nos incluye atributos clave como identificadores únicos de dispositivos y usuarios, coordenadas GPS, nivel de batería, intensidad de señal, detección de manipulación, estado del dispositivo y posibles violaciones de restricciones.

El enfoque de generación aleatoria con restricciones realistas nos permite simular escenarios operativos variados, como dispositivos en movimiento, en reposo o en situaciones de incumplimiento de normas. Además, la inclusión de valores de batería y señal nos permite modelar condiciones de funcionamiento en entornos reales, facilitando el análisis de posibles fallos o pérdidas de conectividad.

Este dataset sintético nos es ideal para poder realizar pruebas, simulaciones y validaciones sin necesidad de recurrir a datos reales, garantizando la privacidad de los usuarios y permitiendo la experimentación con algoritmos de análisis, visualización y predicción en el contexto del monitoreo de activos y seguridad.
